# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [24]:
from nltk.corpus.reader import ConllCorpusReader

# Adapt the path to point to the CONLL2003 folder on your local machine
conll_path = 'CONLL2003\\CONLL2003'
conll_train = ConllCorpusReader(conll_path, 'train.txt', ['words', 'pos', 'ignore', 'chunk'])

In [27]:
#TODO: remove this cell
#This for loop may overwrite earlier 'pos' and 'ne_label' info for tokens that appear multiple times in the 
#text. Does this matter? Consider checking. 

count = 0

for token, pos, ne_label in conll_train.iob_words():
    print(token, pos, ne_label)
    count += 1
    
    if count==10:
        break

EU NNP B-ORG
rejects VBZ O
German JJ B-MISC
call NN O
to TO O
boycott VB O
British JJ B-MISC
lamb NN O
. . O
Peter NNP B-PER


In [28]:
training_features = []
training_gold_labels = []

for token, pos, ne_label in conll_train.iob_words():
    a_dict = {
       'token': token,
       'pos': pos
    }

    training_features.append(a_dict)
    training_gold_labels.append(ne_label)

In [29]:
# Adapt the path to point to the CONLL2003 folder on your local machine
conll_test = ConllCorpusReader(conll_path, 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

In [30]:
test_features = []
test_gold_labels = []

for token, pos, ne_label in conll_test.iob_words():
    a_dict = {
       'token': token,
       'pos': pos
    }

    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [ ]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [40]:
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer()

In [56]:
concatenated_features = training_features + test_features
the_array = vec.fit_transform(concatenated_features)
training_vec, test_vec = the_array[:203621], the_array[203621:]

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [57]:
from sklearn import svm

In [58]:
lin_clf = svm.LinearSVC()

In [59]:
# NOTE: DON'T run this cell again if you've already trained the SVM! It may take a while!
lin_clf.fit(training_vec, training_gold_labels)

c:\Users\kevyn\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

#TODO: remove this Markdown cell <br><br>
NOTE: The above warning might not be critical if the model's performance is acceptable. It is a hint to check whether increasing max_iter or adjusting your model or data preprocessing could improve convergence and possibly lead to a better model.

In [ ]:
# Obtain predictions from the SVM model
svm_predictions = lin_clf.predict(test_vec)

In [62]:
from sklearn.metrics import classification_report

# Generate the classification report and print it
class_report = classification_report(test_gold_labels, svm_predictions)
print(class_report)

              precision    recall  f1-score   support

       B-LOC       0.81      0.77      0.79      1668
      B-MISC       0.78      0.66      0.71       702
       B-ORG       0.79      0.52      0.63      1661
       B-PER       0.87      0.44      0.58      1617
       I-LOC       0.62      0.53      0.57       257
      I-MISC       0.59      0.59      0.59       216
       I-ORG       0.66      0.48      0.55       835
       I-PER       0.33      0.87      0.48      1156
           O       0.99      0.98      0.98     38323

    accuracy                           0.92     46435
   macro avg       0.71      0.65      0.65     46435
weighted avg       0.94      0.92      0.92     46435



**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [10]:
# your code here

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [1]:
import pandas
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm

In [2]:
##### Adapt the path to point to your local copy of NERC_datasets
path = r"C:\Users\Sandy\Documents\GitHub\Text-Mining-Assignments\lab4\ner_v2.csv"
kaggle_dataset = pandas.read_csv(path, on_bad_lines='skip') # Changed error_bad_lines=False to on_bad_lines='skip'

In [3]:
len(kaggle_dataset)

1050795

In [4]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

100000 20000


In [5]:
#TODO: Check if we need only token and POS or all features
training_features = []
training_gold_labels = []

for index, instance in df_train.iterrows():
   token = instance["word"] #token
   pos = instance["pos"]
   ne_label = instance["tag"]
   a_dict = {
       'token': token,
       'pos': pos, 
   }
   training_features.append(a_dict)
   training_gold_labels.append(ne_label)

In [6]:
test_features = []
test_gold_labels = []

for index, instance in df_test.iterrows():
    token = instance["word"] #token
    pos = instance["pos"]       
    ne_label = instance["tag"]
    a_dict = {
        'token': token,
        'pos': pos, 
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)


In [7]:
vec = DictVectorizer()
train_array = vec.fit_transform(training_features).toarray()
print(train_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
# SVM model + training
lin_clf2 = svm.LinearSVC()
lin_clf2.fit(train_array, training_gold_labels)

c:\Users\Sandy\anaconda3\envs\textmining\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

In [10]:
# Testing
test_array = vec.transform(test_features).toarray()
pred = lin_clf2.predict(test_array)

In [11]:
from sklearn.metrics import classification_report
print(classification_report(test_gold_labels, pred))
# NOT CORRECT?

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00         4
       B-eve       0.00      0.00      0.00         0
       B-geo       0.80      0.76      0.78       741
       B-gpe       0.96      0.92      0.94       296
       B-nat       1.00      0.50      0.67         8
       B-org       0.63      0.51      0.57       397
       B-per       0.81      0.53      0.64       333
       B-tim       0.91      0.76      0.83       393
       I-art       0.00      0.00      0.00         0
       I-eve       0.00      0.00      0.00         0
       I-geo       0.74      0.50      0.60       156
       I-gpe       1.00      0.50      0.67         2
       I-nat       0.80      1.00      0.89         4
       I-org       0.66      0.44      0.53       321
       I-per       0.42      0.90      0.57       319
       I-tim       0.41      0.08      0.14       108
           O       0.98      0.99      0.99     16918

    accuracy              

c:\Users\Sandy\anaconda3\envs\textmining\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sandy\anaconda3\envs\textmining\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Sandy\anaconda3\envs\textmining\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TODO: Still need to evaluate performance of SVM on the test data

## End of this notebook